Transform video to audio

In [1]:
from audio_extractor import extract_audio

debra_video = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Soundtrack/[ONTIVA.COM] Debra-144p.mp4"
debra_audio_file = "debra_song"

baby_driver_video_file = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Baby.Driver.2017.720p.BluRay.x264-[YTS.AG].mp4"
baby_driver_audio_file = "baby_driver_audio"

sample_rate = 22000

extract_audio(debra_video, sample_rate, debra_audio_file)
extract_audio(baby_driver_video_file, sample_rate, baby_driver_audio_file)

C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
STARTING: ffmpeg -i C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Soundtrack/[ONTIVA.COM] Debra-144p.mp4 -ac 1 -ar 22000 C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
Audio extracted: C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
debra_song.22000.wav
baby_driver_audio.22000.wav
STARTING: ffmpeg -i C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Baby.Driver.2017.720p.BluRay.x264-[YTS.AG].mp4 -ac 1 -ar 22000 baby_driver_audio.22000.wav


'baby_driver_audio.22000.wav'

Generate Descriptors

In [2]:
from audio_descriptor_generator import create_audio_descriptors

dimension = 32
window = 2200
hop = 2200

baby_driver_audio_file = "baby_driver_audio.22000.wav"
debra_song_audio_file = "debra_song.22000.wav"

debra_descriptors = create_audio_descriptors(debra_song_audio_file, sample_rate, dimension, window, hop)
print(debra_descriptors.shape)
baby_descriptors = create_audio_descriptors(baby_driver_audio_file, sample_rate, dimension, window, hop)
print(baby_descriptors.shape)

baby_driver_descriptors_file = "baby_driver_audio_descriptors.bin"
debra_song_descriptors_file = "debra_song_descriptors.bin"

debra_descriptors.tofile(debra_song_descriptors_file, sep="\n")
baby_descriptors.tofile(baby_driver_descriptors_file, sep="\n")

1.74 seconds
(3434, 32)
(3434, 32)
(67598, 32)


Calculate distances

In [1]:
import numpy
import time
from scipy.spatial import distance

baby_driver_descriptors_file = "baby_driver_audio_descriptors.bin"
# debra_song_descriptors_file = "debra_song_descriptors.bin"
debra_song_descriptors_file = "debra_song_descriptors.bin"

debra_song_shape = (3434, 32)

song_descriptors = numpy.fromfile(debra_song_descriptors_file, sep="\n").reshape(debra_song_shape)
print(song_descriptors.shape)

movie_shape = (67598, 32)
movie_descriptors = numpy.fromfile(baby_driver_descriptors_file, sep="\n").reshape(movie_shape)
print(movie_descriptors.shape)

t0 = time.time()
distances = distance.cdist(song_descriptors, movie_descriptors)
t1 = time.time()

print(f"Distances {distances.shape} {round(t1-t0, 2)} secs")

(3434, 32)
(67598, 32)
Distances (3434, 67598) 14.04 secs


Get neighbours

In [2]:
from duplicate_searcher import Neighbours

number_of_neighbours = 10

neighbour_file = "neighbours.txt"
neighbours = []
total = distances.shape[0]
t10 = time.time()
for i in range(distances.shape[0]):
    print(f"Progress: {round((i/total)*100, 2)}%", end="\r")
    song_descriptor = distances[i]
    neighs = numpy.argpartition(song_descriptor, number_of_neighbours)[:number_of_neighbours]
    neighbours.append(neighs)
t11 = time.time()
neighbours = Neighbours(numpy.array(neighbours))
print(f"Neighbours {neighbours.shape()} {round(t11-t10, 2)} secs")

Neighbours (3434, 10) 45.17 secs


Create Candidates

In [3]:
from duplicate_searcher import Candidate

candidates = []
song_indexes = range(distances.shape[0])
total_songs = distances.shape[0]
for song in song_indexes:
    print(f"Progress {round((song/total_songs)*100, 2)}%", end="\r")
    for neighbour in neighbours.search(song):
        candidates.append(Candidate(song, neighbour, 22000))

print(f" Candidates {len(candidates)}")


 Candidates 34340


Find sequence

In [7]:
print(type(neighbours.search(50)))
print(type(neighbours.search(3434)))
print(f"{21540 in neighbours.search(50)}")

<class 'numpy.ndarray'>
<class 'int'>
True


In [8]:
copies = []

max_missing_streak = 7
min_duration_seconds = 1
sample_rate = 22000
window = 2200
min_duration = 10

total_candidates = len(candidates)
for i in range(len(candidates)):
    print(f"Progress {round((i/total_candidates)*100, 2)}%", end="\r")
    cand = candidates[i]
    current_candidate = cand.find_next(neighbours, max_missing_streak)
    if current_candidate.sequence_duration >= min_duration:
        copies.append(current_candidate)
    
    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()